# Snowpark ML and Snowpark for Python

[Frosty Friday Challenge: Week 59 - Intermediate - Snowpark ML](https://frostyfriday.org/blog/2023/08/18/week-59-intermediate/)

The purpose of this script is to demonstrate simple use of Snowflake's Snowpark ML library. This challenge uses the `DecisionTreeClassifier` constructor, for which we can reference the following [example](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling#example).

## Import the various packages

Before we can begin, we must import the required packages.

In [27]:
import pandas as pd
from snowflake.snowpark.functions import col
from snowflake.snowpark.version import VERSION as snowpark_version

snowpark_version_as_string = ".".join([str(x) for x in list(snowpark_version)])

print(f'Snowpark version: {snowpark_version_as_string}')

Snowpark version: 1.5.1


### Snowpark ML Package

From the Snowpark ML package, we retrieve the `DecisionTreeClassifier` that will be used for this challenge.

In [2]:
from snowflake.ml.modeling.tree import DecisionTreeClassifier

### InterWorks Snowpark package

To simplify creating Snowpark sessions for the enclosed scripts, a custom module called "interworks_snowpark" has been used. This is contained in the "shared/interworks_snowpark" subdirectory. More details can be found in the [InterWorks Snowpark for Python GitHub repository](https://github.com/interworks/InterWorks-Snowpark-for-Python). This repository also contains instructions on how to configure your local environment for Snowpark for Python, and pairs well with this [Definitive Guide to Snowflake Sessions with Snowpark for Python](https://interworks.com/blog/2022/09/02/a-definitive-guide-to-snowflake-sessions-with-snowpark-for-python/).

Alternatively, you can modify the code to establish a Snowflake Snowpark Session through any method of your choice.

In [3]:
## Import module to build snowpark sessions
from shared.interworks_snowpark.interworks_snowpark_python.snowpark_session_builder import build_snowpark_session_via_parameters_json as build_snowpark_session

## Generate Snowpark session
snowpark_session = build_snowpark_session()

## Prepare Snowflake Environment

As we do for all Frosty Friday challenges, we configure our environment for a specific database, schema and warehouse.

In [4]:
snowpark_session.use_database('"CH_FROSTY_FRIDAY"')
snowpark_session.sql("create or replace schema WEEK_59").collect()
snowpark_session.use_schema('"CH_FROSTY_FRIDAY"."WEEK_59"')
snowpark_session.use_warehouse('"WH_CHASTIE"')

current_schema = snowpark_session.get_fully_qualified_current_schema()
current_warehouse = snowpark_session.get_current_warehouse()

print(f'Current schema: {current_schema}')
print(f'Current warehouse: {current_warehouse}')

Current schema: "CH_FROSTY_FRIDAY"."WEEK_59"
Current warehouse: "WH_CHASTIE"


## Ingest the data into Snowflake

Before we can proceed with Snowpark ML, we need some data to play with. This Frosty Friday challenge already has some data for us to leverage. The data we require is stored in the following location:

```s3://frostyfridaychallenges/challenge_59/age_and_income.csv```

We create a stage pointing to this cloud storage location, then ingest the file.

### Create the stage

In [5]:
snowpark_session.sql('''
  create or replace stage "STG_WEEK_59"
    url = 's3://frostyfridaychallenges/challenge_59/'
''').collect()

[Row(status='Stage area STG_WEEK_59 successfully created.')]

### Investigate files in stage

Quickly ingestigate the data to ensure we can ingest it with a standard CSV approach and to learn the table metadata. We could infer schema like we did in challenge 53, but I personally prefer the following method as I can pull in metadata file names and row numbers.

First, ensure the file is in the stage

In [6]:
snowpark_session._list_files_in_stage('@"STG_WEEK_59"')

{'ridaychallenges/challenge_59/age_and_income.csv'}

Create a simple file format that will ingest each record of the data into a single field for simple understanding.

In [7]:
snowpark_session.sql('''
  create or replace file format "FF_SINGLE_FIELD"
    type = CSV
    field_delimiter = NONE
    record_delimiter = '\n'
    skip_header = 0
''').collect()

[Row(status='File format FF_SINGLE_FIELD successfully created.')]

Query the data using the single field file format to understand the contents.

In [33]:
sf_df_single_record = snowpark_session.sql('''
  select
      metadata$filename::string as "FILE_NAME"
    , metadata$file_row_number as "ROW_NUMBER"
    , $1::variant as "CONTENTS"
  from @"STG_WEEK_59"
    (file_format => 'FF_SINGLE_FIELD')
  order by 
      "FILE_NAME"
    , "ROW_NUMBER"
''')

# Show the whole dataframe to ensure it makes sense
sf_df_single_record.show()

# Show only the CONTENTS field for ease of reading
sf_df_single_record.select(col('"CONTENTS"')).show()


-----------------------------------------------------------------------------------------------------
|"FILE_NAME"                      |"ROW_NUMBER"  |"CONTENTS"                                        |
-----------------------------------------------------------------------------------------------------
|challenge_59/age_and_income.csv  |1             |"age,monthly_income,made_purchase,sample_weight"  |
|challenge_59/age_and_income.csv  |2             |"25,3000,0,1"                                     |
|challenge_59/age_and_income.csv  |3             |"30,3200,0,1"                                     |
|challenge_59/age_and_income.csv  |4             |"35,3500,1,1"                                     |
|challenge_59/age_and_income.csv  |5             |"40,3700,0,1"                                     |
|challenge_59/age_and_income.csv  |6             |"45,4000,1,1"                                     |
|challenge_59/age_and_income.csv  |7             |"50,4200,1,1"                   

The above shows us we have the following columns in our data, where we have made an educated guess on the data types:
- AGE - integer
- MONTHLY_INCOME - integer
- MADE_PURCHASE - boolean
- SAMPLE_WEIGHT - integer

### Ingest the data

We can use what we have learned to create a table and ingest the data into it with a standard file format.

Create a standard CSV file format to ingest the data.

In [9]:
snowpark_session.sql('''
  create or replace file format "FF_CSV_INGESTION"
    type = CSV
    field_delimiter = ','
    record_delimiter = '\n'
    skip_header = 1
''').collect()

[Row(status='File format FF_CSV_INGESTION successfully created.')]

Create the table in which we will land the data.

In [10]:
snowpark_session.sql('''
  create or replace table "RAW_DATA" (
      "FILE_NAME" string
    , "ROW_NUMBER" integer
    , "AGE" integer
    , "MONTHLY_INCOME" integer
    , "MADE_PURCHASE" boolean
    , "SAMPLE_WEIGHT" integer
  )
''').collect()

[Row(status='Table RAW_DATA successfully created.')]

Ingest the data.

In [11]:
snowpark_session.sql('''
  copy into "RAW_DATA"
  from (
    select 
        metadata$filename::string as "FILE_NAME"
      , metadata$file_row_number as "ROW_NUMBER"
      , $1::int as "AGE"
      , $2::int as "MONTHLY_INCOME"
      , $3::boolean as "MADE_PURCHASE"
      , $4::int as "SAMPLE_WEIGHT"
    from @"STG_WEEK_59"
      (file_format => 'FF_CSV_INGESTION')
  )
''').collect()

[Row(file='s3://frostyfridaychallenges/challenge_59/age_and_income.csv', status='LOADED', rows_parsed=24, rows_loaded=24, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]

Sample the data to verify successful ingestion.

In [13]:
snowpark_session.table('"RAW_DATA"').show()

-----------------------------------------------------------------------------------------------------------------
|"FILE_NAME"                      |"ROW_NUMBER"  |"AGE"  |"MONTHLY_INCOME"  |"MADE_PURCHASE"  |"SAMPLE_WEIGHT"  |
-----------------------------------------------------------------------------------------------------------------
|challenge_59/age_and_income.csv  |1             |25     |3000              |False            |1                |
|challenge_59/age_and_income.csv  |2             |30     |3200              |False            |1                |
|challenge_59/age_and_income.csv  |3             |35     |3500              |True             |1                |
|challenge_59/age_and_income.csv  |4             |40     |3700              |False            |1                |
|challenge_59/age_and_income.csv  |5             |45     |4000              |True             |1                |
|challenge_59/age_and_income.csv  |6             |50     |4200              |True       

## Snowpark ML Challenge with DecisionTreeClassifier

Now that we have our data in Snowflake, we are ready to begin the main challenge using Snowpark ML. 

### Load the table to a local Pandas dataframe

Load the table into a local pandas dataframe. For simplicity, we select specific fields; excluding metadata fields "FILE_NAME" and "ROW_NUMBER".

In [42]:
# Load the data
df_raw_data = snowpark_session.table('"RAW_DATA"').select(
      col('"AGE"')
    , col('"MONTHLY_INCOME"')
    , col('"MADE_PURCHASE"')
    , col('"SAMPLE_WEIGHT"')
  ).to_pandas()

# Display the contents
display(df_raw_data)

,AGE,MONTHLY_INCOME,MADE_PURCHASE,SAMPLE_WEIGHT
0,25,3000,False,1
1,30,3200,False,1
2,35,3500,True,1
3,40,3700,False,1
4,45,4000,True,1
5,50,4200,True,1
6,55,4500,True,1
7,60,4800,True,1
8,28,3100,False,1
9,33,3300,True,1


### Create the DecisionTreeClassifier model

Leverage `DecisionTreeClassifier` to create a model, for which we can reference the following [example](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling#example).

In [44]:
decision_tree_model = DecisionTreeClassifier(
    input_cols=['AGE','MONTHLY_INCOME']
  , label_cols=['MADE_PURCHASE']
  , sample_weight_col=['SAMPLE_WEIGHT']
  , output_cols=['PREDICTED_PURCHASE']
)

### Fit the model to the data

In [45]:
decision_tree_model.fit(df_raw_data)

### Make predictions using the model

Use the model to generate predicted values.

In [54]:
# Create new dataframe with the predictions
df_with_predictions = decision_tree_model.predict(df_raw_data)

# Display the contents
display(df_with_predictions)

,AGE,MONTHLY_INCOME,MADE_PURCHASE,SAMPLE_WEIGHT,PREDICTED_PURCHASE
0,25,3000,False,1,False
1,30,3200,False,1,False
2,35,3500,True,1,True
3,40,3700,False,1,False
4,45,4000,True,1,True
5,50,4200,True,1,True
6,55,4500,True,1,True
7,60,4800,True,1,True
8,28,3100,False,1,False
9,33,3300,True,1,True


## Challenge Output

For the challenge output, display only the predictions.

In [55]:
display(df_with_predictions[['PREDICTED_PURCHASE']])

,PREDICTED_PURCHASE
0,False
1,False
2,True
3,False
4,True
5,True
6,True
7,True
8,False
9,True
